In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd
import re

In [2]:
vacancies = []
more_page = True

In [3]:
#Функция обработки отдельного блока вакансии
def get_vacancy_info(vacancy):
    #Название вакансии
    vacancy_name = vacancy.find('span',{'class':'resume-search-item__name'}).text
    #Зарплата
    vacancy_salary = vacancy.find('div',{'class':'vacancy-serp-item__sidebar'}).text.replace("\xa0","")
    if not vacancy_salary or vacancy_salary == 'По договоренности':
        salary_min = None
        salary_max = None
        salary_cur = None
    elif re.findall(r'^\w+', vacancy_salary)[0] == 'от':
        salary_min = re.findall(r'\d+', vacancy_salary)[0]
        salary_max = None
        salary_cur = re.findall(r'\w+', vacancy_salary)[-1]
    elif re.findall(r'^\w+', vacancy_salary)[0] == 'до':
        salary_max = re.findall(r'\d+', vacancy_salary)[0]
        salary_min = None
        salary_cur = re.findall(r'\w+', vacancy_salary)[-1]
    else:
        salary_min = re.findall(r'\w+', vacancy_salary)[0]
        salary_max = re.findall(r'\w+', vacancy_salary)[1]
        salary_cur = re.findall(r'\w+', vacancy_salary)[-1]
    
    #Ссылка
    vacancy_url = vacancy.find('span',{'class':'resume-search-item__name'}).a['href']
    return [vacancy_name, salary_min, salary_max, salary_cur, vacancy_url]

In [4]:
#Функция обработки отдельных страниц
def parse_page(url):
    print(f'Parsing url: {url}')
    
    response = requests.get(url,headers=headers)
    soup = bs(response.text, 'lxml')      
    vacancy_all = soup.findAll('div', {'class':'vacancy-serp-item'})
    for vac in vacancy_all:
      vacancies.append(get_vacancy_info(vac))
    try:
        # проверяем есть ли кнопка Далее, и если есть возращаем линк, если нет возращаем False
        new_link = 'https://hh.ru' + soup.find('a',{'class':'HH-Pager-Controls-Next'})['href']
        return new_link
    except:
        return False

In [5]:
job_title = input('Введите название должности: ')
main_link = 'https://hh.ru/search/vacancy'
params = {'text':job_title, 'page' :'0'}
headers = {'User-Agent': 'Mozilla/5.0'}

In [6]:
response = requests.get(main_link,params=params, headers=headers)
soup = bs(response.text, 'lxml')
vacancy_all = soup.findAll('div', {'class':'vacancy-serp-item'})
for vac in vacancy_all:
    vacancies.append(get_vacancy_info(vac))

In [7]:
# Проверяем есть ли клавиша Далее, и даем либо новый линк, либо завершаем
try:
    new_link = 'https://hh.ru' + soup.find('a',{'class':'HH-Pager-Controls-Next'})['href']
except:
    more_page = False
while more_page:
    if not new_link:
        break
    new_link = parse_page(new_link)
     

Parsing url: https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+Science&page=1
Parsing url: https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+Science&page=2
Parsing url: https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+Science&page=3
Parsing url: https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+Science&page=4
Parsing url: https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+Science&page=5
Parsing url: https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+Science&page=6
Parsing url: https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+Science&page=7
Parsing url: https://hh.ru/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+Science&page=8
Parsing 

In [8]:
print(f'Всего найдено {len(vacancies)} вакансий по запросу {job_title}')

Всего найдено 714 вакансий по запросу Data Science


In [9]:
Data = pd.DataFrame(vacancies, columns = ['Название','Зарплата Мин','Зарплата Макс','Валюта','URL'])

In [10]:
Data

,Название,Зарплата Мин,Зарплата Макс,Валюта,URL
0,Директор ИТ,None,None,None,https://hh.ru/vacancy/37364391?query=Data%20Sc...
1,Researcher in Mathematical Modeling and Algori...,None,None,None,https://hh.ru/vacancy/35441422?query=Data%20Sc...
2,UX Researcher,None,None,None,https://hh.ru/vacancy/36703153?query=Data%20Sc...
3,Разработчик DWH [id 21076],None,None,None,https://hh.ru/vacancy/36308035?query=Data%20Sc...
4,Python Data Scientist,150000,None,руб,https://hh.ru/vacancy/36916080?query=Data%20Sc...
...,...,...,...,...,...
709,Middle Android Developer,None,None,None,https://hh.ru/vacancy/36814705?query=Data%20Sc...
710,Fullstack JavaScript разработчик,300000,700000,KZT,https://hh.ru/vacancy/37112096?query=Data%20Sc...
711,Field Coordination Officer / Civil-Military Co...,None,None,None,https://hh.ru/vacancy/37096010?query=Data%20Sc...
712,Junior JavaScript Engineer (Full Stack),None,None,None,https://hh.ru/vacancy/37048363?query=Data%20Sc...
